In [1]:
# Copiar código para ChatGPT

# Códigos Front
archivos_scripts = [
    "app.py",
    "templates/index.html",
    #"templates/serv_translate.html",
    #"templates/serv_translate_progress.html",
    #"templates/serv_translate_result.html",
    #"templates/serv_extract_progress.html",
    #"templates/serv_extract_result.html",
    #"templates/serv_transcribe_progress.html",
    #"templates/serv_edit_progress.html",
    #"templates/serv_summary_progress.html",

    #"templates/serv_translate_result.html",
]

def imprimir_contenido_archivos(lista_archivos):
    for archivo in lista_archivos:
        try:
            with open(archivo, 'r', encoding='utf-8') as file:
                print(f"Contenido de {archivo}:")
                print(file.read())
                print("\n" + "="*40 + "\n")
        except FileNotFoundError:
            print(f"El archivo {archivo} no se encuentra.")
        except IOError as e:
            print(f"Error al leer el archivo {archivo}: {e}")

# Llamar a la función con la lista de archivos
imprimir_contenido_archivos(archivos_scripts)

Contenido de app.py:
from flask import Flask, render_template, request, redirect, url_for, jsonify, send_from_directory, abort
from werkzeug.utils import secure_filename
import os
import threading
import time
from pathlib import Path
import json
from doc_translator import traducir_doc
from model_translator import translate_text
from model_transcribe import transcribe_audio
from doc_editor import editar_doc
from doc_summary import resumir_doc  # Asumimos que tienes una función para resumir documentos
from doc_extract_info import extract_info_from_doc

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = os.path.abspath('docs/uploads')
app.config['RESULT_FOLDER'] = os.path.abspath('docs/downloads')
app.config['ALLOWED_EXTENSIONS'] = {'docx', 'pptx', 'pdf','txt','html', 'm4a', 'wav', 'mp3'}
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16 MB

class FileProcessor:
    def __init__(self, process_func):
        self.process_func = process_func

    def allowed_file(self, filename):
 

In [ ]:
# Copiar código para ChatGPT

# Códigos Front
archivos_scripts = [
    "app.py",
    "doc_translator.py",
]

def imprimir_contenido_archivos(lista_archivos):
    for archivo in lista_archivos:
        try:
            with open(archivo, 'r', encoding='utf-8') as file:
                print(f"Contenido de {archivo}:")
                print(file.read())
                print("\n" + "="*40 + "\n")
        except FileNotFoundError:
            print(f"El archivo {archivo} no se encuentra.")
        except IOError as e:
            print(f"Error al leer el archivo {archivo}: {e}")

# Llamar a la función con la lista de archivos
imprimir_contenido_archivos(archivos_scripts)


In [ ]:
# Prueba Función Traducción

from doc_translator import translate_text

translate_text(texto='Tradueix aquest text',origin_language='auto',destination_language='spanish', add_prompt="" ,model='llama3-70b-8192', api_key_file='API_KEY.txt')

In [ ]:
# Prueba Función Prompting

from model_prompting import prompt_text

prompt_text(texto='Avui he anat al parc que guai, ho fas super ve', add_prompt="Revisa aquest text reescriu-lo amb un to més formal" ,model='llama3-70b-8192', api_key_file='API_KEY.txt')


In [1]:
# Prueba doc_translator.py

from pathlib import Path
from doc_translator import traducir_doc


# VARIABLES:
nombre_fichero = 'test2.docx' # Debe incluir la extensión al final (.pptx, .docx o .pdf)
origin_language = "Catalan"
destination_language = "Spanish"
color_to_exclude = '#FF0000' # poner en formato #F00000

# Rutas de documentos a traducir y devolución
input_path = f'test/in/{nombre_fichero}'
output_path = f'test/out/Traducido_{destination_language}_{nombre_fichero}'
extension = Path(nombre_fichero).suffix


traducir_doc(
            input_path,
            output_path,
            origin_language,
            destination_language,
            extension,
            color_to_exclude,
            None
        )

Starting translation process for test/in/test2.docx
Brúixola 
Estudi sobre 
l'aplicació de la 
intel·ligència
 artificial
202
1
Introducció 
01
01
Introducció i objectius
Dintre del projecte de l'Observatori Nacional de 
Tecnologia i Societat (ONTSI) que té com a principal
L'objectiu és identificar tendències, actituds i valoracions 
de la població internauta espanyola en qüestions 
relacionades amb la digitalització. L'estudi actual analitza 
la percepció de l'opinió pública en relació amb la 
implantació d'elements d'intel·ligència artificial (IA) 
en diversos aspectes relacionats amb la vida quotidiana.
Aquest estudi es realitza dins dels estudis de 
seguiment 
 amb periodicitat semestral, previstos 
en el projecte esmentat anteriorment, que té
Per tal d'abordar l'anàlisi de qüestions d'
actualitat que siguin d'interès en cada moment, 
respecte a elements o aspectes relacionats 
amb el comportament digital de la població.
La intel·ligència artificial, en l'àmbit d'aquest estudi, 
fa

[INFO] HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"



            Follow these rules:
            - Your task is to translate text while strictly preserving codes (_CDTR_00000) in their exact positions, including at the start or end of the text. 
            - Provide only the translated text without any additional comments or annotations. 
            - Ensure that the translation is grammatically correct in Spanish, with special attention to the correct use of apostrophes in articles and pronouns.
            - Translate all words, including those starting with a capital letter, unless they appear to be proper names.
            - Keep similar text length when translating.
            - The text provided is part of a larger document. Use the given context (words before and after) for a more accurate translation.

            Translate the text from Catalan to Spanish.
Text to translate:
(_CDTR_000001)Brúixola (_CDTR_000002)Estudi sobre (_CDTR_000003)l'aplicació de la (_CDTR_000004)intel·ligència(_CDTR_000005) artificial(_CDTR_000008)In

[INFO] HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"



            Follow these rules:
            - Your task is to translate text while strictly preserving codes (_CDTR_00000) in their exact positions, including at the start or end of the text. 
            - Provide only the translated text without any additional comments or annotations. 
            - Ensure that the translation is grammatically correct in Spanish, with special attention to the correct use of apostrophes in articles and pronouns.
            - Translate all words, including those starting with a capital letter, unless they appear to be proper names.
            - Keep similar text length when translating.
            - The text provided is part of a larger document. Use the given context (words before and after) for a more accurate translation.

            Words before the text (use as context):
identificar tendencias, actitudes y valoraciones
Translate the text from Catalan to Spanish.
Text to translate:
identificar tendencias, actitudes y valoraciones (_CDTR_000015

...


            Follow these rules:
            - Your task is to translate text while strictly preserving codes (_CDTR_00000) in their exact positions, including at the start or end of the text. 
            - Provide only the translated text without any additional comments or annotations. 
            - Ensure that the translation is grammatically correct in Spanish, with special attention to the correct use of apostrophes in articles and pronouns.
            - Translate all words, including those starting with a capital letter, unless they appear to be proper names.
            - Keep similar text length when translating.
            - The text provided is part of a larger document. Use the given context (words before and after) for a more accurate translation.

            Words before the text (use as context):
médico. (_CDTR_000087)Inteligencia (_CDTR_000090)inteligencia artificial (IA)
Translate the text from Catalan to Spanish.
Text to translate:
médico. (_CDTR_000087)Inteligenci

[INFO] HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"



            Follow these rules:
            - Your task is to translate text while strictly preserving codes (_CDTR_00000) in their exact positions, including at the start or end of the text. 
            - Provide only the translated text without any additional comments or annotations. 
            - Ensure that the translation is grammatically correct in Spanish, with special attention to the correct use of apostrophes in articles and pronouns.
            - Translate all words, including those starting with a capital letter, unless they appear to be proper names.
            - Keep similar text length when translating.
            - The text provided is part of a larger document. Use the given context (words before and after) for a more accurate translation.

            Words before the text (use as context):
en (_CDTR_000099)el desarrollo(_CDTR_000101) de asistentes(_CD
Translate the text from Catalan to Spanish.
Text to translate:
en (_CDTR_000099)el desarrollo(_CDTR_000101) de 

[INFO] HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"



            Follow these rules:
            - Your task is to translate text while strictly preserving codes (_CDTR_00000) in their exact positions, including at the start or end of the text. 
            - Provide only the translated text without any additional comments or annotations. 
            - Ensure that the translation is grammatically correct in Spanish, with special attention to the correct use of apostrophes in articles and pronouns.
            - Translate all words, including those starting with a capital letter, unless they appear to be proper names.
            - Keep similar text length when translating.
            - The text provided is part of a larger document. Use the given context (words before and after) for a more accurate translation.

            Words before the text (use as context):
aplicación (_CDTR_000112)de la inteligencia artificial
Translate the text from Catalan to Spanish.
Text to translate:
aplicación (_CDTR_000112)de la inteligencia artificial (

In [1]:
from process_text_editor import separar_texto_bloques

# Ejemplo de uso
textos = {
    "1": "Este es un texto de prueba. Está separado en bloques.",
    "2": "Otro texto que también debe ser manejado correctamente. Se probará la separación.",
    "3": "Finalmente, este es el último texto. ¡Gracias por la atención!"
}

bloques = separar_texto_bloques(textos, max_chars_per_block=20)
for i, bloque in enumerate(bloques, 1):
    print(f"Bloque {i}:\n{bloque}\n")


Bloque 1:


Bloque 2:
(_CDTR_1)Este es un texto de prueba. Está separado en bloques.

Bloque 3:
(_CDTR_2)Otro texto que también debe ser manejado correctamente. Se probará la separación.

Bloque 4:
(_CDTR_3)Finalmente, este es el último texto. ¡Gracias por la atención!



In [27]:
import fitz  # PyMuPDF

def convertir_text_a_majuscules(text):
    return text.upper()

def rgb_to_pymupdf_color(color):
    if isinstance(color, (tuple, list)) and len(color) == 3:
        return tuple(c / 255 for c in color)  # Convertir a rang de 0 a 1
    elif isinstance(color, int):
        return ((color >> 16) & 255) / 255, ((color >> 8) & 255) / 255, (color & 255) / 255
    else:
        print(f"Format de color no reconegut: {color}. Utilitzant negre per defecte.")
        return (0, 0, 0)

def processar_pdf(input_pdf, output_pdf):
    doc = fitz.open(input_pdf)
    
    for page_num in range(len(doc)):
        page = doc[page_num]
        
        redactions = []
        new_texts = []
        
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:
            if b["type"] == 0:  # Text block
                for line in b["lines"]:
                    for span in line["spans"]:
                        text_majuscules = convertir_text_a_majuscules(span["text"])
                        bbox = fitz.Rect(span["bbox"])
                        
                        # Afegir redaccions i text nou
                        redactions.append(bbox)
                        new_texts.append({
                            "text": text_majuscules,
                            "bbox": bbox,
                            "font": "helv",  # Utilitzar una font per defecte
                            "size": span["size"],
                            "color": rgb_to_pymupdf_color(span["color"])
                        })
        
        # Aplicar les redaccions per eliminar el text original
        for bbox in redactions:
            page.add_redact_annot(bbox)
        page.apply_redactions()
        
        # Inserir el nou text
        for text_info in new_texts:
            # Ajustar la posició del text per centrar-lo dins del bbox
            text_bbox = text_info["bbox"]
            text_rect = fitz.Rect(text_bbox.tl.x, text_bbox.tl.y, text_bbox.br.x, text_bbox.br.y)
            text_height = text_info["size"]  # Alçada de la font
            text_top = text_bbox.tl.y + (text_bbox.height - text_height) / 2  # Centrar verticalment

            page.insert_text(
                (text_bbox.tl.x, text_top),
                text_info["text"],
                fontname=text_info["font"],
                fontsize=text_info["size"],
                color=text_info["color"]
            )
    
    # Guardar el nou PDF
    doc.save(output_pdf)
    doc.close()

# Especifiquem el PDF d'entrada i el de sortida
input_pdf = "test/in/pdf_largo.pdf"
output_pdf = "test/out/sortida_majuscules.pdf"

# Processem el PDF per crear-ne una còpia amb tot el text en majúscules
processar_pdf(input_pdf, output_pdf)


In [1]:
# Prueba doc_summarize:


from doc_summary import resumir_doc
nombre_fichero = 'pdf_largo.pdf'

input_path = f'test/in/{nombre_fichero}'

resumir_doc(input_path, num_words=100, summary_language='catalan', add_prompt="")


"L'estudi analitza la percepció de la població espanyola respecte a la implementació de la intel·ligència artificial (IA) en diferents àmbits de la vida quotidiana. L'objectiu és avaluar les opinions sobre la implementació de la IA en àmbits com asistents personals, vehicles i salut. El resultat és que hi ha una percepció dual en funció del tipus de desenvolupament, amb un major interès en l'ús de la IA en vehicles i assistents virtuals, mentre que la seva aplicació en el camp mèdic genera menys interès i certa desconfiança en la seva seguretat.\nL'interès pel desenvolupament d'aplicacions d'Intelligència Artificial (IA) és elevat entre la població, especialment en àrees com l'automòbil, on el 55% mostra molt interès. Els asistents virtuales també generen un alt interès, amb un 51% que els considera interessants. Per contra, l'aplicació de la IA en el àmbit mèdic genera menys confiança, amb un 31% que desconfia de la seva implementació.\nResum de l'estudi sobre l'aplicació de la intel·

In [2]:
# Prueba con doc_extract_info

from doc_extract_info import extract_info_from_doc

file = 'test/in/pdf_largo.pdf'

prompts = ['El documento está en español?', '¿De qué año es el estudio?', 'Explicame en 5 palabras de que va el documento']
tipos_respuesta = ['SI/NO', 'numérica','texto libre']
ejemplos_respuesta = ['SI', '2025','Estudio sobre las ranas del lago']  # Esto es opcional

resultados = extract_info_from_doc(file, prompts, tipos_respuesta, ejemplos_respuesta)


Procesando archivo: test/in/pdf_largo.pdf
Dividiendo texto en 1 bloques.
Procesando bloque de texto...
El documento está en español?
['SI']
¿De qué año es el estudio?
['2021']
Explicame en 5 palabras de que va el documento
['Estudio sobre aplicación de la inteligencia artificial.']
{'El documento está en español?': 'SI', '¿De qué año es el estudio?': '2021', 'Explicame en 5 palabras de que va el documento': 'Estudio sobre aplicación de inteligencia artificial.'}


In [1]:
# Prueba doc_editor.py

from pathlib import Path
from doc_editor import editar_doc


# VARIABLES:
nombre_fichero = 'TEST.pdf' # Debe incluir la extensión al final (.pptx, .docx o .pdf)

color_to_exclude = '#FF0000' # poner en formato #F00000

# Rutas de documentos a traducir y devolución
input_path = f'test/in/{nombre_fichero}'
output_path = f'test/out/Editado_{nombre_fichero}'
extension = Path(nombre_fichero).suffix

add_prompt = 'Pon todo el texto en mayúsculas'

editar_doc(
            input_path,
            output_path,
            extension,
            color_to_exclude,
            add_prompt
        )

[INFO] Start to convert test/in/TEST.pdf
[INFO] [1/4] Opening document...
[INFO] [2/4] Analyzing document...


Starting translation process for test/in/TEST.pdf


[INFO] [3/4] Parsing pages...
[INFO] (1/4) Page 1
[INFO] (2/4) Page 2
[INFO] (3/4) Page 3
[INFO] (4/4) Page 4
[INFO] [4/4] Creating pages...
[INFO] (1/4) Page 1
[INFO] (2/4) Page 2
[INFO] (3/4) Page 3
[INFO] (4/4) Page 4
[INFO] Terminated in 0.75s.


Diccionario textos_originales
{'000001': 'Tema 1.  Especificidades para la evaluación y ', '000002': 'diagnóstico en adultos en el ámbito sanitario ', '000003': 'Todo diagnóstico y evaluación psicológico debe enmarcarse en:', '000004': 'Otro tipo de dificultades deberán volver a ser evaluadas. Si los nuevos problemas no están relacionados, ', '000005': 'debemos hacer otra evaluación.', '000006': 'Los orígenes de un problema son importantes, pero nosotros solo podemos trabajar en el momento ', '000007': 'presente. ', '000008': '¿', '000009': 'Qué es EVALUAR', '000010': '? valorar y medir.', '000011': 'Evaluar tiene un fin, un propósito, nos lleva a algo, que es comprender y conocer como funciona algo. Lo ', '000012': 'importante de evaluar es poder predecir y conocer como se comportaría alguien en una situación. ', '000013': 'La evaluación es un proceso compuesto por fases sucesivas e interrelacionadas.', '000014': 'Evaluación y diagnóstico como parte de la intervención en Psicología Ge

KeyboardInterrupt: 